### Imports

In [30]:
from pyod.models.ecod import ECOD
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.47 ms (started: 2023-06-10 08:25:03 -05:00)


### Parameters

In [31]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = params["method"]
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = params["option"]
print("Option:\t\t", option)

if option:
    
    subgroups_id_op = {}
    for group in groups_id:
        subgroups_id_op[group] = [option]
else:
    subgroups_id_op = subgroups_id
print("Subgroups id op:", subgroups_id_op)



Exp:		 exp1
Method:		 dgi
Dimension:	 3
Groups id:	 ['WT', 'zwf1^', 'pck1^']
Subgroups id:	 {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
Option:		 dyn
Subgroups id op: {'WT': ['dyn'], 'zwf1^': ['dyn'], 'pck1^': ['dyn']}
time: 4.18 ms (started: 2023-06-10 08:25:03 -05:00)


### Edge embeddings

In [32]:
# get edges embeddings

edge_embeddings_global(exp, method, groups_id, subgroups_id_op)

100%|██████████| 3/3 [45:03<00:00, 901.33s/it] 

time: 45min 4s (started: 2023-06-10 08:25:03 -05:00)


In [33]:
df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, groups_id[0], subgroups_id_op[groups_id[0]][0]), index_col=[0, 1])
df_edge_embeddings.head()

0             1         2
0 1  0.000623  1.468307e-07  0.001172
  2  0.000002  7.043716e-10  0.000004
  3  0.000964  1.502997e-07  0.001826
  4  0.000131  1.575025e-10  0.000256
  5  0.000228  6.943225e-08  0.000426

time: 4.47 s (started: 2023-06-10 09:10:07 -05:00)


### Concat edge embeddings

In [34]:
for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.DataFrame()
    k = 0
    for subgroup in tqdm(subgroups_id_op[group]):
        df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, group, subgroup), index_col=[0, 1])
        df_edge_embeddings["subgroup"] = [k] * len(df_edge_embeddings)
        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
        k += 1
    
    df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index=True)

100%|██████████| 3/3 [03:39<00:00, 73.15s/it]

time: 3min 39s (started: 2023-06-10 09:10:12 -05:00)


In [35]:
df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, groups_id[0], option), index_col=[0, 1])
df_edge_embeddings_concat.head()

0             1         2  subgroup
0 1  0.000623  1.468307e-07  0.001172         0
  2  0.000002  7.043716e-10  0.000004         0
  3  0.000964  1.502997e-07  0.001826         0
  4  0.000131  1.575025e-10  0.000256         0
  5  0.000228  6.943225e-08  0.000426         0

time: 4.56 s (started: 2023-06-10 09:13:51 -05:00)


In [36]:
# plot edge embeddings concat

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    x = df_edge_embeddings_concat.iloc[:, 0]
    y = df_edge_embeddings_concat.iloc[:, 1]
    z = df_edge_embeddings_concat.iloc[:, 2]

    # Creating figure
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")

    # Creating plot
    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)
    # plt.title("Dimension: {}".format(dimension))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_concat_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

100%|██████████| 3/3 [13:35<00:00, 271.83s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 13min 35s (started: 2023-06-10 09:13:56 -05:00)


### Outliers detection

In [37]:
# Outlier detection (HDBSCAN)

""" df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

X_train = df_edge_embeddings_concat.iloc[:, :-1]
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)

threshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]
print(len(outliers))
outliers

inliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)
print(len(inliers))
inliers """

' df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])\n\nX_train = df_edge_embeddings_concat.iloc[:, :-1]\nclusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)\n\nthreshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)\noutliers = np.where(clusterer.outlier_scores_ > threshold)[0]\nprint(len(outliers))\noutliers\n\ninliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)\nprint(len(inliers))\ninliers '

time: 4.19 ms (started: 2023-06-10 09:27:32 -05:00)


In [38]:
# outlier detection (ECOD)

dict_df_edge_embeddings_concat_outlier = {}
dict_df_edge_embeddings_concat_filter = {}

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    X_train = df_edge_embeddings_concat.iloc[:, :-1]

    clf = ECOD()
    clf.fit(X_train)

    X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
    df_edge_embeddings_concat_filter["labels"] = clf.labels_
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

    dict_df_edge_embeddings_concat_outlier[group] = X_train
    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:55<00:00, 18.49s/it]

time: 55.6 s (started: 2023-06-10 09:27:32 -05:00)


In [39]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
0 1  0.000623  1.468307e-07  0.001172         0
  3  0.000964  1.502997e-07  0.001826         0
  5  0.000228  6.943225e-08  0.000426         0
  7  0.002065  4.558815e-07  0.003888         0
  8  0.003653  3.862374e-07  0.006963         0

time: 12.8 ms (started: 2023-06-10 09:28:28 -05:00)


In [40]:
# plot outliers/inliers

for group in tqdm(groups_id):
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")

    df_aux = dict_df_edge_embeddings_concat_outlier[group]
    print("Total:", len(df_aux))
    
    temp = df_aux[df_aux["labels"] == 0]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="red", alpha=0.005)
    print("Num. of inliers:", len(temp))

    temp = df_aux[df_aux["labels"] == 1]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="gray", alpha=0.005)
    print("Num. of inliers:", len(temp))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_outlier_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

  0%|          | 0/3 [00:00<?, ?it/s]

Total: 12257101
Num. of inliers: 11031391
Num. of inliers: 1225710


 33%|███▎      | 1/3 [04:30<09:00, 270.13s/it]

Total: 17101443
Num. of inliers: 15391298
Num. of inliers: 1710145


 67%|██████▋   | 2/3 [10:58<05:39, 339.79s/it]

...

100%|██████████| 3/3 [13:18<00:00, 266.32s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 13min 18s (started: 2023-06-10 09:28:28 -05:00)


###  Filter common edges

In [41]:
print(subgroups_id)
print(subgroups_id_op)


{'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
{'WT': ['dyn'], 'zwf1^': ['dyn'], 'pck1^': ['dyn']}
time: 741 µs (started: 2023-06-10 09:41:47 -05:00)


In [42]:
# mapping idx with id

for group in tqdm(groups_id):
    dict_df_nodes = {}
    for subgroup in subgroups_id_op[group]:
        df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup),
                               dtype={"id": "string"})
        dict_df_nodes[subgroup] = df_nodes
    
    # mapping
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
    list_index = []
    
    for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
        df_nodes = dict_df_nodes[subgroups_id_op[group][row[-1]]]
        list_index.append((df_nodes.iloc[row[0][0], -1], df_nodes.iloc[row[0][1], -1]))
    
    # set new index
    df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)

11031391it [04:53, 37522.38it/s]it/s]
15391298it [06:50, 37524.37it/s]6, 303.32s/it]
5445463it [02:24, 37771.27it/s]12, 372.84s/it]
100%|██████████| 3/3 [14:33<00:00, 291.17s/it]

time: 14min 33s (started: 2023-06-10 09:41:47 -05:00)


In [43]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
A59.0049 A274.0123  0.000623  1.468307e-07  0.001172         0
         A369.1213  0.000964  1.502997e-07  0.001826         0
         A503.0516  0.000228  6.943225e-08  0.000426         0
         A558.4273  0.002065  4.558815e-07  0.003888         0
         A641.1223  0.003653  3.862374e-07  0.006963         0

time: 13.3 ms (started: 2023-06-10 09:56:20 -05:00)


In [44]:
# format id
if option:
    for group in tqdm(groups_id):
        # format
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        list_index = []

        for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
            list_index.append((row[0][0][1:], row[0][1][1:]))
        
        # set new index
        df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)
        df_edge_embeddings_concat_filter

11031391it [00:09, 1132091.28it/s]/s]
15391298it [00:13, 1133698.85it/s] 20.11s/it]
5445463it [00:04, 1222027.82it/s], 24.47s/it]
100%|██████████| 3/3 [00:57<00:00, 19.21s/it]

time: 57.6 s (started: 2023-06-10 09:56:20 -05:00)


In [45]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0             1         2  subgroup
59.0049 274.0123  0.000623  1.468307e-07  0.001172         0
        369.1213  0.000964  1.502997e-07  0.001826         0
        503.0516  0.000228  6.943225e-08  0.000426         0
        558.4273  0.002065  4.558815e-07  0.003888         0
        641.1223  0.003653  3.862374e-07  0.006963         0

time: 13.9 ms (started: 2023-06-10 09:57:18 -05:00)


In [46]:
# filter diferente edges

if option:
    for group in tqdm(groups_id):
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter.index.get_level_values(0) != df_edge_embeddings_concat_filter.index.get_level_values(1)]
        dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:03<00:00,  1.09s/it]

time: 3.27 s (started: 2023-06-10 09:57:19 -05:00)


In [47]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
print(df_edge_embeddings_concat_filter.shape)
df_edge_embeddings_concat_filter.head()

(11008321, 4)


0             1         2  subgroup
59.0049 274.0123  0.000623  1.468307e-07  0.001172         0
        369.1213  0.000964  1.502997e-07  0.001826         0
        503.0516  0.000228  6.943225e-08  0.000426         0
        558.4273  0.002065  4.558815e-07  0.003888         0
        641.1223  0.003653  3.862374e-07  0.006963         0

time: 9.84 ms (started: 2023-06-10 09:57:22 -05:00)


In [48]:
# count edges and filter by count

dict_df_edges_filter = {}
for group in tqdm(groups_id):
    # count
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]

    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["level_0", "level_1"]].value_counts().to_frame()
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]

    # filter
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
    dict_df_edges_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:13<00:00,  4.60s/it]

time: 13.8 s (started: 2023-06-10 09:57:22 -05:00)


In [49]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,818.5519,908.4906
1,745.5002,963.5293
2,645.4494,808.5671
3,645.4494,808.5342
4,645.4494,808.5047


time: 9.55 ms (started: 2023-06-10 09:57:36 -05:00)


In [50]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28600 entries, 0 to 28599
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  28600 non-null  object
 1   target  28600 non-null  object
dtypes: object(2)
memory usage: 670.3+ KB
time: 22.8 ms (started: 2023-06-10 09:57:36 -05:00)


In [51]:
# change data type
for group in tqdm(groups_id):
    df_edges_filter = dict_df_edges_filter[group]
    df_edges_filter[["source", "target"]] = df_edges_filter[["source", "target"]].astype("string")

100%|██████████| 3/3 [00:00<00:00, 25.02it/s]

time: 125 ms (started: 2023-06-10 09:57:36 -05:00)


In [52]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,818.5519,908.4906
1,745.5002,963.5293
2,645.4494,808.5671
3,645.4494,808.5342
4,645.4494,808.5047


time: 10.8 ms (started: 2023-06-10 09:57:36 -05:00)


In [53]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28600 entries, 0 to 28599
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  28600 non-null  string
 1   target  28600 non-null  string
dtypes: string(2)
memory usage: 670.3 KB
time: 16.6 ms (started: 2023-06-10 09:57:37 -05:00)


In [54]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

 67%|██████▋   | 2/3 [00:53<00:27, 27.77s/it]

In [ ]:
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
20545,109.0407,114.056,0.684680,0.654750,0.527725,0.859966,0.823646
20623,109.0407,127.0513,0.791174,0.792589,0.782065,0.917810,0.870420
19976,109.0407,131.0462,0.802158,0.721020,0.613603,0.872517,0.867231
19970,109.0407,131.0824,0.808212,0.663848,0.719276,0.857543,0.913011
19991,109.0407,132.086,0.764665,0.614573,0.603960,0.893537,0.878882


time: 13 ms (started: 2023-06-10 02:42:16 -05:00)


### Filter by STD and average weight

In [ ]:
dict_df_common_edges = std_global(dict_df_edges_filter_weight, exp, method, groups_id, option, th=0.3, plot=True, save=True)
dict_df_common_edges[groups_id[0]].head()

100%|██████████| 3/3 [00:13<00:00,  4.55s/it]


,source,target,weight
0,109.0407,114.056,0.710154
1,109.0407,127.0513,0.830812
2,109.0407,131.0462,0.775306
3,109.0407,131.0824,0.792378
4,109.0407,132.086,0.751123


<Figure size 640x480 with 0 Axes>

time: 13.7 s (started: 2023-06-10 02:42:16 -05:00)


In [ ]:
df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, groups_id[0], option),
                              dtype={"source": "string", "target": "string"})
df_common_edges.head()

,source,target,weight
0,109.0407,114.056,0.710154
1,109.0407,127.0513,0.830812
2,109.0407,131.0462,0.775306
3,109.0407,131.0824,0.792378
4,109.0407,132.086,0.751123


time: 28 ms (started: 2023-06-10 02:42:30 -05:00)


In [ ]:
# show details

for group in tqdm(groups_id):
    df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, group, option))
    
    G = nx.from_pandas_edgelist(df_common_edges, "source", "target", edge_attr=["weight"])
    print("Group: {}".format(group))
    graph_detail(G)

  0%|          | 0/3 [00:00<?, ?it/s]

Group: WT
Num. nodes: 1917
Num. edges: 23319



 67%|██████▋   | 2/3 [00:00<00:00,  2.01it/s]

Group: zwf1^
Num. nodes: 4364
Num. edges: 522116



100%|██████████| 3/3 [00:01<00:00,  1.61it/s]

Group: pck1^
Num. nodes: 5045
Num. edges: 454537

time: 1.87 s (started: 2023-06-10 02:42:30 -05:00)
